In [2]:
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import pickle

In [3]:
with open("documents.pkl", "rb") as f:
    documents = pickle.load(f)

In [4]:
embed = OllamaEmbeddings(
    model="mxbai-embed-large"
)

In [5]:
vectorstore = FAISS.load_local('faiss_index_constitution', embed, allow_dangerous_deserialization=True)
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
bm25_retriever = BM25Retriever.from_documents(documents)


In [6]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever],
                                       weights=[0.15, 0.85])

In [16]:
ensemble_retriever.get_relevant_documents('Who is the Head of State of Singapore?')

[Document(metadata={'Legislation': 'Constitution of the Republic of Singapore', 'Part No.': 'PART 5', 'Part Title': 'THE GOVERNMENT', 'Section Title': 'The President', 'Subsection No.': '17.', 'id': '67'}, page_content='Constitution of the Republic of Singapore PART 5 THE GOVERNMENT The President 17.<br><br>17.—(1)There shall be a President of Singapore who shall be the Head of State.[28/2016](2)In addition to being the Head of State, it is also the function of the President to safeguard the reserves of Singapore and the integrity of the Public Services of Singapore, and the President is to perform this function according to the provisions of this Constitution mentioned in clause(3).[28/2016](3)The provisions mentioned by'),
 Document(id='933db892-1dbc-40cc-aa84-4e19791a3b0b', metadata={'Legislation': 'Constitution of the Republic of Singapore', 'Part No.': 'PART 6', 'Part Title': 'THE LEGISLATURE', 'Section Title': 'Legislature of Singapore', 'Subsection No.': '62.', 'id': '403'}, pag

In [17]:
vectorstore.similarity_search_with_relevance_scores('''Who is the Head of State of Singapore?''',k=10)

[(Document(id='965fde98-ba1e-4ba7-824e-8388169adcd0', metadata={'Legislation': 'Constitution of the Republic of Singapore', 'Part No.': 'PART 5', 'Part Title': 'THE GOVERNMENT', 'Section Title': 'The President', 'Subsection No.': '17.', 'id': '67'}, page_content='Constitution of the Republic of Singapore PART 5 THE GOVERNMENT The President 17.<br><br>17.—(1)There shall be a President of Singapore who shall be the Head of State.[28/2016](2)In addition to being the Head of State, it is also the function of the President to safeguard the reserves of Singapore and the integrity of the Public Services of Singapore, and the President is to perform this function according to the provisions of this Constitution mentioned in clause(3).[28/2016](3)The provisions mentioned by'),
  np.float32(0.6493315)),
 (Document(id='933db892-1dbc-40cc-aa84-4e19791a3b0b', metadata={'Legislation': 'Constitution of the Republic of Singapore', 'Part No.': 'PART 6', 'Part Title': 'THE LEGISLATURE', 'Section Title':

In [22]:
ensemble_retriever.get_relevant_documents('What is the process for dissolving Parliament?')

[Document(id='b76e1873-3c21-4c52-a163-52ff38f33eb3', metadata={'Legislation': 'Constitution of the Republic of Singapore', 'Part No.': 'PART 6', 'Part Title': 'THE LEGISLATURE', 'Section Title': 'Legislature of Singapore', 'Subsection No.': '65.', 'id': '413'}, page_content='a reasonable period has elapsed since that office was last vacated and that there is no Member of Parliament likely to command the confidence of a majority of the Members thereof.(3)The President may, at any time, by Proclamation in the Gazette, dissolve Parliament if he is advised by the Prime Minister to do so, but he shall not be obliged to act in this respect in accordance with the advice of the Prime Minister unless he is satisfied that, in tendering that advice, the Prime Minister commands the'),
 Document(id='ac006ad9-7a53-4f08-a48d-e97a92917e62', metadata={'Legislation': 'Constitution of the Republic of Singapore', 'Part No.': 'PART 6', 'Part Title': 'THE LEGISLATURE', 'Section Title': 'Legislature of Singa

In [23]:
vectorstore.similarity_search_with_relevance_scores('''What is the process for dissolving Parliament?''',k=10)

[(Document(id='b76e1873-3c21-4c52-a163-52ff38f33eb3', metadata={'Legislation': 'Constitution of the Republic of Singapore', 'Part No.': 'PART 6', 'Part Title': 'THE LEGISLATURE', 'Section Title': 'Legislature of Singapore', 'Subsection No.': '65.', 'id': '413'}, page_content='a reasonable period has elapsed since that office was last vacated and that there is no Member of Parliament likely to command the confidence of a majority of the Members thereof.(3)The President may, at any time, by Proclamation in the Gazette, dissolve Parliament if he is advised by the Prime Minister to do so, but he shall not be obliged to act in this respect in accordance with the advice of the Prime Minister unless he is satisfied that, in tendering that advice, the Prime Minister commands the'),
  np.float32(0.5985091)),
 (Document(id='ac006ad9-7a53-4f08-a48d-e97a92917e62', metadata={'Legislation': 'Constitution of the Republic of Singapore', 'Part No.': 'PART 6', 'Part Title': 'THE LEGISLATURE', 'Section T

In [18]:
# # Now retrieve the actual documents using those IDs
# documents = [vectorstore.docstore._dict[doc_id] for doc_id in vectorstore.index_to_docstore_id.values()]

# # Example: Print each document chunk
# for i, doc in enumerate(documents):
#     print(f"Chunk {i+1}:\n{doc.page_content}\n{'-'*50}")
#     if i == 10:
#         break